## API Key Input

In [1]:
import os
import getpass

# Re-enter Tavily API Key
tavily_key = getpass.getpass("🔑 Enter Tavily API Key: ")
os.environ["TAVILY_API_KEY"] = tavily_key

# Re-enter OpenAI API Key
openai_key = getpass.getpass("🔑 Enter OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_key


🔑 Enter Tavily API Key:  ········
🔑 Enter OpenAI API Key:  ········


##  Load Tavily Search Tool

In [ ]:
#Load the Tavily web search tool from langchain_tavily.Set it up to return max 2 results.
#tool.invoke(...) checks if the tool works by asking a test question.

In [2]:
from langchain_tavily import TavilySearch

# Define web search tool
tool = TavilySearch(max_results=2)
tools = [tool]

# Test if Tavily works
tool.invoke("What is LangGraph?")



{'query': 'What is LangGraph?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
   'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
   'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactio

## Initialize ChatOpenAI and Tool Binding

In [25]:
#Purpose:
#Load OpenAI’s GPT-3.5 model for conversation.
#bind_tools(tools) connects GPT with the Tavily tool, so GPT can ask for web searches if needed.

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm_with_tools = llm.bind_tools(tools)


##  Define Graph State Type

In [ ]:
#Define the data format (state) to pass between nodes in LangGraph.
#Each state contains a list of messages exchanged in the chat.

In [5]:
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

# Define the state type
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize the stateful graph
graph_builder = StateGraph(State)


##  Define Chatbot Node 

In [26]:
#Define how the chatbot responds to the current message.
#It uses OpenAI model (GPT) + tools to generate a reply.

In [6]:
# Define chatbot node logic
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Add chatbot node to graph
graph_builder.add_node("chatbot", chatbot)


## Tool Execution Node

In [27]:
#Define how to handle tool usage, e.g., Tavily web search.
#If GPT requests a tool, this node executes it and sends back a message with the tool output.

In [7]:
import json
from langchain_core.messages import ToolMessage

# Tool execution node
class BasicToolNode:
    def __init__(self, tools: list):
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        messages = inputs.get("messages", [])
        if not messages:
            raise ValueError("No message found in input")
        last_message = messages[-1]
        outputs = []
        for tool_call in getattr(last_message, "tool_calls", []):
            result = self.tools_by_name[tool_call["name"]].invoke(tool_call["args"])
            outputs.append(
                ToolMessage(
                    content=json.dumps(result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

# Add tool node
tool_node = BasicToolNode(tools)
graph_builder.add_node("tools", tool_node)


## Router Logic for Next Node

In [ ]:
#Decide where to go next in the graph:
#If GPT wants to use a tool ➝ go to tools node.
#If no tool is needed ➝ end the conversation.
#After tool use ➝ go back to chatbot.

In [8]:
# Routing function
def router(state: State):
    messages = state.get("messages", [])
    if not messages:
        return "chatbot"
    last_msg = messages[-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools"
    return "end"

# Add edges based on routing
graph_builder.add_conditional_edges("chatbot", router, {
    "tools": "tools",
    "end": "end"
})
graph_builder.add_edge("tools", "chatbot")


## Add End Node and Set Entry/Exit Points

In [ ]:
#Purpose:
#Add a dummy end node to mark the end of a conversation.
#Set the start point as chatbot and end point as end.
#Compile the whole graph and print the structure.

In [9]:
# ✅ Add this dummy end node BEFORE defining the finish point
def end_node(state: State):
    return {}

graph_builder.add_node("end", end_node)

# ✅ Define entry and finish points
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("end")

# ✅ Compile the graph
app = graph_builder.compile()


In [10]:
#!pip install grandalf==0.7


In [11]:
app.get_graph().print_ascii()


      +-----------+       
      | __start__ |       
      +-----------+       
            *             
            *             
            *             
      +---------+         
      | chatbot |         
      +---------+         
        ..     ..         
       .         .        
      .           .       
+-------+       +-----+   
| tools |       | end |   
+-------+       +-----+   
                    *     
                    *     
                    *     
              +---------+ 
              | __end__ | 
              +---------+ 


## Define Chat Function

In [ ]:
#Define how to send user input to the graph and stream the assistant’s reply.
#Collects assistant output and prints it.

In [28]:
# Stream chat from the graph
def stream_chat(user_input: str):
    full_response = ""  # accumulate assistant reply
    for event in app.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            for msg in value["messages"]:
                full_response += msg.content  # collect chunks

    print("Assistant:", full_response)  # print once



## Start Chat Loop

In [30]:
#Purpose:
#Create a loop to chat with the assistant.
#If the user types exit, it stops.
#If there’s an error, it prints it and stops.

In [29]:
# Interactive chat loop
while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            print("👋 Goodbye!")
            break
        stream_chat(user_input)
    except Exception as e:
        print("❌ Error:", e)
        break

You:  Give some motivational quotes


Assistant: {"query": "motivational quotes", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.shopify.com/blog/motivational-quotes", "title": "200+ Motivational Quotes for Success & Daily Inspiration (2025)", "content": "1.   \u201cWe cannot solve problems with the kind of thinking we employed when we came up with them.\u201d \u2014Albert Einstein\n2.   \u201cLearn as if you will live forever, live like you will die tomorrow.\u201d \u2014Mahatma Gandhi\n3.   \u201cStay away from those people who try to disparage your ambitions. Small minds will always do that, but great minds will give you a feeling that you can become great too.\u201d \u2014Mark Twain [...] 1.   \u201cJust one small positive thought in the morning can change your whole day.\u201d \u2014Dalai Lama\n2.   \u201cOpportunities don\u2019t happen, you create them.\u201d \u2014Chris Grosser\n3.   \u201cLove your family, work super hard, live your passion.\u201d \u2014Gary Vaynerchuk\n

You:  ok thankyou


Assistant: You're welcome! If you have any more questions or need assistance with anything else, feel free to ask. Have a great day!


You:  q


👋 Goodbye!
